In [1]:
import csv, requests, json, collections, os
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import re
regex = re.compile('[^a-zA-Z0-9-_]')
import xlwt 
from xlwt import Workbook
import pandas as pd
from os import listdir
from os.path import isfile, join

In [2]:
category_properties = {}
category_properties['Hardware-Devices'] = ['notRelevant','isPowered','isConnected','isSetup','isUsed']
category_properties['Software-Device-Drivers'] = ['notRelevant','isInstalled','isRelatedDeviceConnected','isSettingsChanged']
category_properties['Software-OS-Related'] = ['notRelevant','isOpened','isSettingsChanged']
category_properties['Software-Other'] = ['notRelevant','isInstalled','isOpened','isSettingsChanged']
category_properties['Hardware-Other'] = ['']

property_event = {}
property_event['notRelevant'] = True
property_event['isPowered'] = False
property_event['isConnected'] = False
property_event['isSetup'] = True
property_event['isUsed'] = True
property_event['isInstalled'] = False
property_event['isRelatedDeviceConnected'] = False
property_event['isSettingsChanged'] = True
property_event['isOpened'] = False

property_values = {}
property_values[True] = ['true','false']
property_values[False] = ['f->t','t->f','noChange','start-T','start-F']

In [3]:
step_details = []
mentions = {}
entities = {}
filter_prop = 'isOpened'
f = open('./annotations/state_change_annotations_'+filter_prop+'.tsv','w',encoding='utf-8')

In [4]:
def process_word(word):
    w = word
    cap = False
    if(w.isupper()):
        cap = True
    w = w.lower()
#     print(w)
    if('\'s' == w):
        w=''
    if('cd' == w):
        w='CD'
    w = lemmatizer.lemmatize(w)
#     print(w)
    if(not cap):
        w = w.capitalize()
    else:
        w = w.upper()
    w = regex.sub('', w)
    if(len(w)<2 and w!= 'X'):
        w=''
    return w

def get_line(offset):
    for i, step in enumerate(step_details):
        if(int(offset)<step[2]):
            return i-2
    return len(step_details)-1
def clean_text(ent_text):
    ent_words = ent_text.split(' ')
    ent_words = [process_word(word) for word in ent_words]
    ent_text = ' '.join(ent_words).strip()
    return ent_text
def get_text(mention):
    text = mentions[mention][-2]
    while(entities[text][0] != '' and entities[text][0] != text):
        text = entities[text][0]
    return text
def get_base_ent(text):
    while(entities[text][0] != '' and entities[text][0] != text):
        text = entities[text][0]
    return text

In [10]:
def parse_annotations(doc_id=1,folder='config/',file='monitor',filter_prop=''):
    global entities, mentions, step_details
    config_file = folder+file+'.ann'
    text_file = folder+file+'.txt'
    annotations = open(config_file,'r',encoding='utf-8').readlines()
    raw_text = open(text_file,'r',encoding='utf-8').read().strip()
    steps = raw_text.split('\n\n')
    
    step_details = []
    start = 0
    start2 = 0
    for i, step in enumerate(steps):
        length = len(step)
        newlines = step.count('\n')
        step_details.append([i,step,start,start2,length,newlines])
        start+=length+newlines+4 ### accounting for '\n' diff in Brat
        start2+=2+length
    #     if(i==3):
    #         print(step,'\n\n',newlines)
    num_steps = len(step_details) - 1
    
    entities = {} #base-entity, mentions
    mentions = {} #ent_id, line_no, ent_cat, start, end, ent_text, properties
    for line in annotations:
        line=line.strip()
        if(line[0]=='T'): ### entities
            ent_id, ent_cat, ent_text = line.split('\t')
            ent_text = clean_text(ent_text)
            ent_cat, start, end = ent_cat.split(' ')
            line_no = get_line(start)
            start,end,line_no = int(start),int(end),int(line_no)
            mentions[ent_id] = [ent_id, line_no,ent_cat,start,end,ent_text,{}]
            if(ent_text not in entities):
                entities[ent_text]=['',[]]
            entities[ent_text][1].append(ent_id)
    #         print(ent_id,ent_cat,start,end,line_no,ent_text,sep='\n')
    #         break
        elif(line[0]=='A'): ### attributes
            atr_id, atr_type = line.split('\t')
            split = atr_type.split(' ')
            if(len(split)==2):
                split.append('true')
            atr_type, atr_ent, atr_val = split
            mentions[atr_ent][-1][atr_type]=atr_val
    #         print(atr_id,atr_type, atr_ent, atr_val,sep='\n')
    #         break
        elif(line[0]=='*'): ### equiv
            _, rel_type = line.split('\t')
            rels = rel_type.split(' ')
            rel_type, ent1, ent2 = rels[0], rels[1], rels[2]
            text1, text2 = get_text(ent1), get_text(ent2)
            if(text1==text2):
                continue
            base, other = text2, text1
            if(len(text1)<=len(text2)):
                base, other = text1, text2
            entities[other][0]=base
    #         print(entities[other][0],base,sep=',')
        elif(line[0]=='R'): ### relation- subpart
            rel_id, rel_type = line.split('\t')
            rel_type, sub, main = rel_type.split(' ')
            sub, main = sub.split(':')[1],main.split(':')[1]
            sub, main = get_text(sub), get_text(main)
            entities[sub][0] = main
    #         print(rel_id,rel_type,sub,main,sep=',\t')
        else:
            print(line)
    
    merged_ents = {} ### merge entities with base-subpart-equiv
    for ent in entities:
        base_ent = get_base_ent(ent)
        if(base_ent not in merged_ents):
            merged_ents[base_ent] = {}
        for mention_id in entities[ent][1]:
            mention = mentions[mention_id]
            line_no = mention[1]
            if(line_no not in merged_ents[base_ent]):
                merged_ents[base_ent][line_no] = mention
            else:
                merged_ents[base_ent][line_no][-1].update(mention[-1])

    clean_ents = []
    for ent in merged_ents:
        if(all(merged_ents[ent][line_no][-1]=={} for line_no in merged_ents[ent])):
            clean_ents.append(ent)
    for ent in clean_ents:
        del merged_ents[ent]
        
    step_wise_properties = {}

    for ent in merged_ents:
        mention = mentions[entities[ent][1][0]]
        category = mention[2]
        if(category == 'Hardware-Other'):
            continue
        properties = category_properties[category]
        ent_properties = {}
        for prop in properties:
            prop_val = ['']*(num_steps+1)
            prop_val[0]='False'
    #         print(sorted(merged_ents[ent]))
    #         break
    #         ent_mentions = [mentio]
    #         ent_mentions = sorted(merged_ents[ent])
            if(property_event[prop]):
                for step_no in range(0,num_steps):
                    if(step_no in merged_ents[ent] and prop in merged_ents[ent][step_no][-1]):
                        val = merged_ents[ent][step_no][-1][prop].capitalize()
                        prop_val[step_no+1] = val
                    else:
                        prop_val[step_no+1] = 'False'
            else:
                for step_no in range(0,num_steps):
                    if(step_no in merged_ents[ent] and prop in merged_ents[ent][step_no][-1]):
                        val = merged_ents[ent][step_no][-1][prop]
                        if(val=='f->t' and prop_val[step_no]=='False'):
                            prop_val[step_no+1] = 'True'
                        elif(val=='t->f' and prop_val[step_no]=='True'):
                            prop_val[step_no+1] = 'False'
                        elif(val=='t->f' and all(val=='False' for val in prop_val[:step_no])):
                            for i in range(step_no+1):
                                prop_val[i] = 'True'
                            prop_val[step_no+1] = 'False'
                        elif(val=='start-T'):
                            for i in range(step_no+2):
                                prop_val[step_no+1] = 'True'
                        elif(val=='start-F'):
                            for i in range(step_no+2):
                                prop_val[step_no+1] = 'False'
                        else:
                            prop_val[step_no+1] = prop_val[step_no]
                    else:
                        prop_val[step_no+1] = prop_val[step_no]
            if not((filter_prop != '' and prop!=filter_prop) or all(val=='False' for val in prop_val)):
                ent_properties[prop]=prop_val
    #     print(ent,category,ent_properties,sep='\n')
    #     break
        step_wise_properties[ent]=ent_properties
    # print(step_wise_properties)

    df = pd.DataFrame({'Steps':steps})
    for ent in step_wise_properties:
        for prop in step_wise_properties[ent]:
            if(filter_prop==''):
                df[ent+'___'+prop] = step_wise_properties[ent][prop]
            else:
                df[ent] = step_wise_properties[ent][prop]
    df.to_excel('./config/'+file+'.xlsx')
    
#     headers = list(df.columns)[1:]
#     if(len(headers)==0):
#         print('\nSKIPPING',doc_id,folder,file,filter_prop,sep=',\t')
#         return
#     print(doc_id,'SID','PARTICIPANTS','\t'.join(headers),sep='\t',file=f)
#     print(doc_id,'','PROMPT: '+steps[0].split('\n')[0],'\t'.join(['-=====' for head in headers]),sep='\t',file=f)
#     print(doc_id,'state1','','\t'.join(list(df.iloc[0].values)[1:]),sep='\t',file=f)
#     for step_no in range(num_steps):
#         states = list(df.iloc[step_no+1].values)
#         print(doc_id,'event'+str(step_no+1),repr(states[0])[1:-1],sep='\t',file=f)
#         print(doc_id,'state'+str(step_no+2),'','\t'.join(states[1:]),sep='\t',file=f)
#     print('\n',file=f)
#     f.close()

In [15]:
parse_annotations(folder='./config/',file='how_to_setup_printer')
# parse_annotations(folder='./annotations/OS/',file='224_1___Format_a_PC___Formatting_Your_Primary_Drive')
# parse_annotations(folder='./annotations/Headphones/',file='163_1___Connect_a_Headset_to_PC___Connecting_via_Bluetooth')


In [7]:
# parse_annotations()
# f.close()

In [8]:
# data_folder = './annotations/'
# count = 1
# for folder in [fol for fol in listdir(data_folder) if not isfile(join(data_folder, fol))]:
#     folder = data_folder + folder+'/'
#     onlyfiles = [f[:-4] for f in listdir(folder) if f.endswith('.txt')]
#     for file in onlyfiles:
#         print(count, folder, file)
#         parse_annotations(doc_id=count, folder=folder, file=file, filter_prop=filter_prop)
#         count+=1

In [9]:
f.close()